# Inference Mask RCNN

Load model from [mmdetection framework](https://github.com/open-mmlab/mmdetection) and process some test images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import onnxruntime as ort
import cv2
import typing as tp

In [ ]:
IMAGE_DIR = '../test_images'
NN_THRESHOLD = 0.7
BATCH_SIZE=1

In [ ]:
def onnx_preprocessing(
    image: np.ndarray,
    image_size: tp.Tuple[int, int] = (224, 224),
) -> np.ndarray:
    # resize
    image = cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_LINEAR)

    # normalize
    mean = np.array((0.485, 0.456, 0.406), dtype=np.float32) * 255.0
    std = np.array((0.229, 0.224, 0.225), dtype=np.float32) * 255.0
    denominator = np.reciprocal(std, dtype=np.float32)
    image = image.astype(np.float32)
    image -= mean
    image *= denominator

    # transpose
    image = image.transpose((2, 0, 1))[None]
    return image


def binary_mask_iou(
        mask1: np.array,
        mask2: np.array,
        label_value: int = 1,
) -> float:
    mask1_area = np.count_nonzero(mask1 == label_value)
    mask2_area = np.count_nonzero(mask2 == label_value)
    intersection = np.count_nonzero(
        np.logical_and(mask1==label_value,  mask2==label_value)
    )
    return intersection / (mask1_area + mask2_area-intersection)

In [ ]:
import json
from tqdm import tqdm

# Process annotations data
d_path_annot = '../IoU_test/annotations/balanced_test.json'
d_path_images = '../IoU_test/images'

processed_data = {
    'Id': [],
    'image_path': [],
    'semantic_masks': [],
    'granules_number': []
}

with open(d_path_annot, 'r', encoding="utf-8") as json_file:
    json_data_dir = json.load(json_file)
    # Process image data
    for image_inf in tqdm(json_data_dir['images'], desc="Process images: "):
        real_img_id = image_inf['id']
        for k in processed_data:
            processed_data[k].append([])
        processed_data['Id'][-1] = real_img_id
        img_name = image_inf['file_name'].split('/')[-1]
        img_path = os.path.join(
            d_path_images, img_name
        )
        processed_data['image_path'][-1] = str(img_path)
        SIZE = (image_inf['height'], image_inf['width'], 3)
        processed_data['semantic_masks'][-1] = np.zeros(SIZE[:2], dtype=np.uint8)
        processed_data['granules_number'][-1] = 0

with open(d_path_annot, 'r') as json_file:
    json_data_dir = json.load(json_file)
    image_id_old = ''
    skipped_counter = 1
    morph_kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3, 3)
    )

    for annotation_data in tqdm(
        json_data_dir['annotations'], desc="Process annotations: "
    ):
        # Data may be numerated to image of have through numeration
        process_image_id = annotation_data['image_id']

        image_data_indx = processed_data['Id'].index(process_image_id)
        label = annotation_data['category_id']
        # Process each granule
        for point_i, point in enumerate(annotation_data['segmentation']):
            if isinstance(point, list):
                point_xy = [
                    [point[j], point[j + 1]] for j in
                    range(0, len(point), 2)
                ]
                cnt = np.array(point_xy).reshape((-1, 1, 2)).astype(
                    np.int32
                )
                if len(cnt) < 3:  # bad contour
                    print('Cnt is bad')
                    continue

                single_mask = np.zeros((480, 480), dtype=np.uint8)
                _ = cv2.drawContours(
                    single_mask,
                    [cnt], -1, label, cv2.FILLED
                )
                processed_data['semantic_masks'][image_data_indx][single_mask != 0] = label
                x, y, w, h = annotation_data["bbox"]
                processed_data['granules_number'][image_data_indx] += 1
            else:
                continue

In [ ]:
providers = [
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    '../models/MaskRCNN-ResNet50.onnx',
    providers=providers
)

In [ ]:
for image_path in list(os.walk(IMAGE_DIR + '/preprocess/'))[0][2]:
    test_image = cv2.imread(IMAGE_DIR + '/preprocess/' + image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    
    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    boxes_scores, labels, masks = ort_session.run(None, ort_inputs)
    
    bboxes = boxes_scores[0,:,:4] # raw bounding boxes
    scores = boxes_scores[0,:,4]   # scores
    labels = labels[0,:]           # raw labels
    masks = np.transpose(masks, [1, 0, 2, 3])
    
    green_masks = np.zeros(test_image.shape, dtype=np.uint8)
    for bbox_i, bbox in enumerate(bboxes):
        if scores[bbox_i] >= NN_THRESHOLD:
            _ = cv2.rectangle(
                    test_image, 
                    (int(bbox[0]), int(bbox[1])),
                    (int(bbox[2]), int(bbox[3])),
                    (255,0,0),
                    2
                )
            bbox = [int(b) for b in bbox]
            mask = masks[bbox_i][0]
            mask = cv2.resize(mask, (bbox[2]-bbox[0]+1, bbox[3]-bbox[1]+1))
            mask = mask > NN_THRESHOLD
            
            im_mask = np.zeros((test_image.shape[0], test_image.shape[1]), dtype=np.uint8)
            x_0 = max(bbox[0], 0)
            x_1 = min(bbox[2] + 1, test_image.shape[1])
            y_0 = max(bbox[1], 0)
            y_1 = min(bbox[3] + 1, test_image.shape[0])
            mask_y_0 = max(y_0 - bbox[1], 0)
            mask_y_1 = mask_y_0 + y_1 - y_0
            mask_x_0 = max(x_0 - bbox[0], 0)
            mask_x_1 = mask_x_0 + x_1 - x_0
            
            im_mask[y_0:y_1, x_0:x_1] = mask[
                mask_y_0 : mask_y_1, mask_x_0 : mask_x_1
            ]
            
            green_masks[:,:,0][im_mask>0] = 61
            green_masks[:,:,1][im_mask>0] = 142
            green_masks[:,:,2][im_mask>0] = 48
    
    test_image = cv2.addWeighted(test_image, 0.6, green_masks, 0.4, 0)
    cv2.imwrite(IMAGE_DIR + f'/final/maskrcnn_{int(NN_THRESHOLD*10)}_' + image_path, cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    
    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.imshow(test_image, cmap='gray')

In [ ]:
total_iou = []
total_granules_det = []

for image_i, image_path in enumerate(tqdm(processed_data['image_path'])):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    gt_mask = processed_data['semantic_masks'][image_i]

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    boxes_scores, labels, masks = ort_session.run(None, ort_inputs)

    bboxes = boxes_scores[0,:,:4] # raw bounding boxes
    scores = boxes_scores[0,:,4]   # scores
    labels = labels[0,:]           # raw labels
    masks = np.transpose(masks, [1, 0, 2, 3])

    pr_mask = np.zeros(test_image.shape[:2], dtype=np.uint8)
    for bbox_i, bbox in enumerate(bboxes):
        if scores[bbox_i] >= NN_THRESHOLD:
            bbox = [int(b) for b in bbox]
            mask = masks[bbox_i][0]
            mask = cv2.resize(mask, (bbox[2]-bbox[0]+1, bbox[3]-bbox[1]+1))
            mask = mask > NN_THRESHOLD

            im_mask = np.zeros((test_image.shape[0], test_image.shape[1]), dtype=np.uint8)
            x_0 = max(bbox[0], 0)
            x_1 = min(bbox[2] + 1, test_image.shape[1])
            y_0 = max(bbox[1], 0)
            y_1 = min(bbox[3] + 1, test_image.shape[0])
            mask_y_0 = max(y_0 - bbox[1], 0)
            mask_y_1 = mask_y_0 + y_1 - y_0
            mask_x_0 = max(x_0 - bbox[0], 0)
            mask_x_1 = mask_x_0 + x_1 - x_0

            im_mask[y_0:y_1, x_0:x_1] = mask[
                mask_y_0 : mask_y_1, mask_x_0 : mask_x_1
            ]
            pr_mask[im_mask>0] = 1

    total_granules_det.append(len(bboxes))
    total_iou.append(binary_mask_iou(gt_mask, pr_mask))

granules_deviation = np.abs((np.array(total_granules_det) - np.array(processed_data['granules_number']))) / np.array(processed_data['granules_number']) * 100
granules_detected = (np.array(total_granules_det) / np.array(processed_data['granules_number'])) * 100


print(f"Median and Average IoU for all dataset = {np.median(total_iou)} VS {np.average(total_iou)}, disp = {np.std(total_iou)}")
print(f"Median and Average granules devianion for all dataset = {np.median(granules_deviation)} VS {np.average(granules_deviation)}, disp = {np.std(granules_deviation)}")
print(f"Median and Average granules detected for all dataset = {np.median(granules_detected)} VS {np.average(granules_deviation)}, disp = {np.std(granules_detected)}")